In [ ]:
!pip install transformers datasets torch accelerate


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [ ]:
with open('data.txt', 'w') as f:
    f.write('This is the first line of text.\n')
    f.write('This is the second line of text.\n')
    f.write('This is the third line of text.\n')

from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "data.txt"})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-textgen",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=6, training_loss=4.454620997111003, metrics={'train_runtime': 19.421, 'train_samples_per_second': 0.463, 'train_steps_per_second': 0.309, 'total_flos': 587907072000.0, 'train_loss': 4.454620997111003, 'epoch': 3.0})

In [ ]:
import torch

prompt = "Artificial intelligence is"
inputs = tokenizer(prompt, return_tensors="pt")

# Move input_ids and attention_mask to the same device as the model
# In Colab, the model is usually on 'cuda' if a GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = {k: v.to(device) for k, v in inputs.items()}

output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"], # Pass attention mask explicitly
    max_length=80,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id # Explicitly set pad token id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Artificial intelligence is in fact the only thing that's going to be able to bring the world to a better.

So that's the kind of thing that's been said about it a lot.


But the thing is it's not just a technology. It's not just a technology. It's a computer.
